# Gas Lift Optimization Notebook

This notebook was originally created for a SPE Online Education Course by two PE undergraduate students from The University of Tulsa, Blazej Ksiazek (ksiazeklblazej@gmail.com) and Antonio Reinoso (amc7606@utulsa.edu). For this course, it has been significantly enhanced by Vinicius Kramer Scariot.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import all necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import callbacks
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MaxAbsScaler

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model

import os.path
from os import path
import keras
import pandas as pd

# Number of inputs, epochs to train on and outputs

In [ ]:
num_inputs = 2  
nb_epochs = 200  
num_output = 1  

# Building the Artificial Neural Network

In [ ]:
model = Sequential()
model.add(Dense(4, batch_input_shape=(None, num_inputs), activation="tanh"))
model.add(Dense(4, activation="tanh"))
model.add(Dense(num_output, activation="tanh"))
model.compile(loss="MSE", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

# Load and preprocess training data

In [ ]:
data = pd.read_csv('GL_Optimization_data.txt', delimiter='\t', header=None, names=['pressure', 'Q_injected', 'Q_produced'])

In [ ]:
data

In [ ]:
data_train = data.sample(frac=0.7)
data_test = data.loc[data.index.difference(data_train.index)]

In [ ]:
raw_train_inputs = data_train[['pressure', 'Q_injected']].values
raw_test_inputs = data_test[['pressure', 'Q_injected']].values

# Scale each feature by its maximum absolute value

In [ ]:
inputScaler = MaxAbsScaler().fit(raw_train_inputs)
inputs = inputScaler.transform(raw_train_inputs)
inputs_test = inputScaler.transform(raw_test_inputs)

# raw_train_outputs

In [ ]:
raw_train_outputs = data_train[['Q_produced']].values
raw_test_outputs = data_test[['Q_produced']].values
outputScaler = MaxAbsScaler().fit(raw_train_outputs)

outputs = outputScaler.transform(raw_train_outputs)
outputs_test = outputScaler.transform(raw_test_outputs)

test_data = tf.data.Dataset.from_tensors((inputs_test, outputs_test))

# Check if model has been trained before. If the model has not been trained before, train it and save it to the disk in a file called "saved_model"

In [ ]:
history = model.fit(x=inputs, y=outputs, epochs=nb_epochs, batch_size=num_output, shuffle=True, validation_data=test_data)
model.save("saved_model.hdf5")

# makes a graph of the training performance
fig = plt.figure()
plt.title('Training performance')
plt.loglog(history.epoch, history.history['loss'], label='train loss')
plt.loglog(history.epoch, history.history['val_loss'], label='test loss')
plt.legend()
fig.savefig("NNTraining.png")


# Obtain normalized model prediction and denormalize

In [ ]:
p = np.linspace(200, 2600, 300)
Q_inj = np.linspace(0, 2, 300)
p, Q_inj = np.meshgrid(p, Q_inj)
inputs = np.array([p.ravel(), Q_inj.ravel()]).T
inputs_prediction = inputScaler.transform(inputs)
normalized_prediction = model.predict(inputs_prediction)
prediction = outputScaler.inverse_transform(normalized_prediction)

# Creates plot of actual Qinj and predicted Qinj of our model

In [ ]:
fig, (ax, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
shape = (56, 6)
pressure_levels = [0, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750]
X, Y = data['pressure'].values.reshape(shape), data['Q_injected'].values.reshape(shape)
Z =  data['Q_produced'].values.reshape(shape)
contour = ax.contourf(Y, Z, X, cmap='Blues', vmin=0, vmax=3000, levels=pressure_levels)
cbar = plt.colorbar(contour, ax=ax)
cbar.set_label('Pressure')
ax.set_xlabel('Injected flow rate')
ax.set_ylabel('Produced flow rate')

contour = ax2.contourf(Q_inj, prediction.reshape(Q_inj.shape), p, cmap='Blues', levels=pressure_levels)
cbar = plt.colorbar(contour, ax=ax2)
cbar.set_label('Pressure')
ax2.set_xlabel('Injected flow rate')
ax2.set_ylabel('Predicted produced flow rate by NN')
ax.set_ylim([0, 1.75])

In [ ]:
fig, ax = plt.subplots()
data_plot = data.loc[[p in [1200, 1500, 1800, 2000] for p in data['pressure']]]
data_plot.groupby('pressure').plot('Q_injected', 'Q_produced', ax=ax)
ax.legend(['1200 psi', '1500 psi', '1800 psi', '2000 psi'])

# Calculates error of our program

In [ ]:
error_normalized = model.evaluate(inputs_test, outputs_test, verbose=False)
error = outputScaler.inverse_transform(np.array(error_normalized).reshape(-1, 1))
error